In [12]:
import pandas as pd
import cbpro
from datetime import datetime, timedelta

# Make using PublicClient Methods easier
public_client = cbpro.PublicClient()

In [13]:
# Get list of crypto currencies available on CoinbasePro
currencies = public_client.get_currencies()

currency_ids = []

for currency in currencies:
    currency_ids.append(currency['id'])

print(f"Number of currencies: {len(currency_ids)}")
currency_ids[:10]

Number of currencies: 97


['ALGO', 'DASH', 'OXT', 'ATOM', 'KNC', 'MIR', 'REP', 'ICP', 'CGLD', 'COMP']

The CoinbasePro API only allows for 300 candles maximum per call to get_product_historic_rates().
Therefore, we will need to perform multiple calls to get data from all days where the crypto 
was available to trade. After some searching, GDAX (former name for Coinbase Pro) was launched in 2015
and Ethereum was launched in 2015 as well. So we will start at 1/1/2015 and work up to today's date 


In [14]:
# Create function that allows for parameterization during the API call
def get_end_date(start_date,days):
    """ Given a start date in datetime format, 
        calculate the end date in datetime format
        based on the number of days provided."""
    
    end_date = start_date + timedelta(days=days)
    return end_date

In [15]:
# Containers
historical_data = {}
symbol = []
time = []
low = []
high = []
opn = []
close = []
volume = []

# Parameters
start = '2015-01-01'
time_delta = 300
granularity = 86400 #Daily candles
today = datetime.now()

# Pull the data and save to DataFrame
for c in currency_ids:
    
    start_date = datetime.strptime(start,'%Y-%m-%d')
    
    while start_date <= today:
        
        end_date = get_end_date(start_date,time_delta)
        
        # Adjust for end_date being in the future
        if end_date > today:
            end_date = today

        historic_rates = public_client.get_product_historic_rates(
            f"{c}-USD", 
            start=datetime.strftime(start_date,'%Y-%m-%d'), 
            end=datetime.strftime(end_date,'%Y-%m-%d'), 
            granularity=granularity
        )

        # Handle case where there is no data available
        # Return from API will be in dict format if no data is availble
        if type(historic_rates) is not dict: 
            try:
                for day in historic_rates:
                    symbol.append(c)
                    time.append((datetime.fromtimestamp(day[0]) + timedelta(days=1)))
                    low.append(day[1])
                    high.append(day[2])
                    opn.append(day[3])
                    close.append(day[4])
                    volume.append(day[5])
            except Exception as e:
                print('c')
                print(day[0])
                print(e)

        # Calculate new start date to use
        start_date = end_date + timedelta(days=1)

# Create DataFrame
data = {
    'symbol':symbol,
    'date':time,
    'high':high,
    'low':low,
    'open':opn,
    'close':close,
    'volume':volume
}
df = pd.DataFrame(data)

In [ ]:
df[df['Symbol'] == 'ETH'].sort_values('Date').head()

,Symbol,Date,High,Low,Open,Close,Volume
15316,ETH,2016-05-18 19:00:00,14.93,12.50,12.50,13.18,482.521827
15315,ETH,2016-05-19 19:00:00,14.90,13.00,13.18,14.90,950.441205
15314,ETH,2016-05-20 19:00:00,14.82,13.71,14.82,14.17,254.166617
15313,ETH,2016-05-23 19:00:00,13.91,13.61,13.86,13.61,180.786712
15312,ETH,2016-05-24 19:00:00,13.74,12.00,13.68,12.77,2753.239298


In [18]:
df.to_csv('/Volumes/develop/projects/crypto-analysis/output/cbpro_data.csv', index=False)

In [27]:
historic_rates = public_client.get_product_historic_rates(
            f"BTC-USD", 
            start='2021-08-09', 
            end=datetime.strftime(datetime.now(),'%Y-%m-%d'), 
            granularity=granularity
        )
historic_rates

[[1629417600, 46644.95, 49400, 46769.02, 49091.58, 12435.93465307],
 [1629331200, 43955, 47088.08, 44709.1, 46765.87, 11938.12185013],
 [1629244800, 44216.47, 46035.27, 44675.34, 44707.98, 10327.49887365],
 [1629158400, 44405.39, 47174.62, 45903.22, 44671.58, 12756.14777945],
 [1629072000, 45676.91, 48076.52, 47014.38, 45902.46, 10467.66031475],
 [1628985600, 45525.19, 47400, 47109.64, 47014.49, 7745.70200802],
 [1628899200, 46034.93, 48176.52, 47833.64, 47109.65, 8130.36769388],
 [1628812800, 44249.64, 47953.9, 44422.15, 47833.64, 11794.94196257],
 [1628726400, 43714.36, 46223.95, 45553.1, 44422.14, 13169.78176994],
 [1628640000, 45350.74, 46781.09, 45601.82, 45553.49, 11628.32043954],
 [1628553600, 44650.27, 46718.44, 46280, 45595.66, 13756.50066875],
 [1628467200, 42821.3, 46497.42, 43829.15, 46285.48, 19464.72493503]]

In [31]:
datetime.fromtimestamp(historic_rates[-1][0])+ timedelta(days=1)

datetime.datetime(2021, 8, 8, 19, 0)

In [26]:
datetime.now()

datetime.datetime(2021, 8, 20, 16, 49, 19, 491544)